In [5]:
search_for = 121
start_from = 0
threads = 10
thread = 6
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004099130630493164
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 15
all cases: 237758115
type: [1, 1, 1, 1, 121] 121
cases of this type: 214358881
100000 11.00252325110466
200000 18.619809914018592
300000 17.601439627943158
400000 17.5989205446299
500000 18.8268061698638
600000 20.018147822662044
700000 17.29891616649337
800000 17.831717347606496
900000 10.102876167411189
1000000 15.390018097997542
1100000 17.873206433612314
1200000 18.59166508297899
1300000 17.57551295903578
1400000 18.594434187439997
1500000 17.562794517623693
1600000 17.454522429835627
1700000 17.711302405339982
1800000 9.111965378982534
1900000 16.49346045850787
2000000 21.089576023931137
2100000 22.626864288236884
2200000 20.976130390010294
2300000 19.789912215028423
2400000 20.400482987998306
2500000 20.96898883717583
2600000 19.105845714236253
2700000 8.028410832683136
2800000 20.377463731384942
2900000 19.128818009684693
3000000 19

29400000 13.832450882263412
29500000 15.04761220167192
29600000 14.93983693724319
29700000 15.94440903348139
29800000 17.114871894191946
29900000 9.252352423466368
30000000 15.38600261970064
30100000 15.832205613290041
30200000 15.789201085477911
30300000 15.555185005014845
30400000 8.353346930275649
30500000 15.73257986472362
30600000 15.689402758607299
30700000 15.20485149228971
30800000 15.141551171774562
30900000 14.900823215120434
31000000 10.40397635404942
31100000 10.024181445939094
31200000 14.948259488919307
31300000 15.14354611935701
31400000 15.13691365629328
31500000 15.720129081326109
31600000 16.61364564689432
31700000 8.738495366353558
31800000 15.264305833750875
31900000 15.821638492891731
32000000 16.714996930300508
32100000 16.309502518148694
32200000 7.553646084183193
32300000 17.63432163544168
32400000 16.43669648838664
32500000 15.2440780101323
32600000 15.15030475642987
32700000 14.545373334003301
32800000 6.885975165060811
32900000 14.08972945478368
33000000 14.4

59100000 11.726873361421903
59200000 12.153056151016129
59300000 11.282604758315033
59400000 5.146693020618032
59500000 11.846651966226178
59600000 11.361558223369666
59700000 11.4300416047436
59800000 6.346672540785992
59900000 12.016806392826272
60000000 12.00217351383253
60100000 13.335377562022142
60200000 12.766859356973129
60300000 13.243276533848753
60400000 13.562199338630188
60500000 13.230490974604024
60600000 12.796823222806962
60700000 13.072466091055714
60800000 5.73498721781637
60900000 11.330640976030617
61000000 12.27998015316527
61100000 10.168871321193514
61200000 6.785638877472708
61300000 11.539804523225897
61400000 11.596961916276658
61500000 8.647616094191134
61600000 9.434806741995182
61700000 12.50683091104295
61800000 12.462116835632802
61900000 12.830561001459298
62000000 13.132737010148576
62100000 12.915589038265505
62200000 13.425312541992255
62300000 12.770489055183583
62400000 12.212648597682596
62500000 11.42715438238196
62600000 6.421832721517772
627000

88800000 11.032975589010785
88900000 10.087921468926362
89000000 9.82362681254469
89100000 10.953783465770215
89200000 10.490078015018327
89300000 9.116010737351083
89400000 6.22259754788044
89500000 4.0862522195820015
89600000 8.754843322003053
89700000 6.749615095006634
89800000 10.461423090765509
89900000 10.127492831987297
90000000 10.2302589954831
90100000 10.088501465055437
90200000 10.474084929820359
90300000 10.424476862605333
90400000 11.507904517299798
90500000 11.520631599130004
90600000 9.848328668420763
90700000 10.584277985709818
90800000 10.08788994294551
90900000 9.740187793560624
91000000 10.669274694167674
91100000 7.570840381946478
91200000 7.926812343456029
91300000 4.349810829574545
91400000 5.742126725002193
91500000 9.574247963319916
91600000 10.12856424150905
91700000 10.321040266129971
91800000 10.215483839104097
91900000 9.58654750751347
92000000 10.502150375682106
92100000 11.012611201686948
92200000 11.068068686726768
92300000 10.644824937705435
92400000 10.

118100000 8.272760190096031
118200000 8.301801942970247
118300000 8.162820843452526
118400000 8.140460680493113
118500000 8.14714877156361
118600000 8.128238864427033
118700000 9.177302139940007
118800000 8.841519344553669
118900000 8.610729969110531
119000000 7.829059768194689
119100000 8.0565222442891
119200000 7.977079511160184
119300000 7.643603728745156
119400000 8.194698383837869
119500000 4.663346082900355
119600000 3.6237710102808576
119700000 4.135972028285037
119800000 7.147553601008763
119900000 8.180401518296552
120000000 8.118953953300833
120100000 8.46288341628543
120200000 8.039961175366697
120300000 7.996085865133514
120400000 8.161140516904526
120500000 8.615985820276698
120600000 8.8481069037412
120700000 8.239765595224698
120800000 8.887117038466275
120900000 7.988095484317155
121000000 8.142175849465337
121100000 8.52427092120754
121200000 7.726847225833992
121300000 4.91791005442062
121400000 3.548992562054399
121500000 4.443829984767685
121600000 8.024136016817113

147400000 5.95848183097581
147500000 6.010390320678158
147600000 4.0140329728564925
147700000 4.874703147054235
147800000 5.864304340096744
147900000 5.408569892416239
148000000 3.0862346120976074
148100000 6.004998456511557
148200000 6.017327547920714
148300000 2.8042833383928105
148400000 6.0340137157728675
148500000 6.636691045560092
148600000 6.311136936369889
148700000 6.549704069690837
148800000 6.546779680712164
148900000 6.982147010759251
149000000 6.518563854927119
149100000 6.470709535901437
149200000 6.26975868904375
149300000 6.038388792104877
149400000 3.396097007127702
149500000 5.740103596356226
149600000 5.932067612919639
149700000 4.323677269408762
149800000 4.412002354562521
149900000 5.991902914467709
150000000 6.515844702405581
150100000 3.086341054225865
150200000 6.067225375846594
150300000 6.377998768071343
150400000 6.466004880996555
150500000 6.6851504360294935
150600000 6.690903514447422
150700000 6.40657178408438
150800000 6.580792112831622
150900000 6.158948

176600000 4.379407929489424
176700000 4.335799272788525
176800000 4.711309382641865
176900000 2.315326532745815
177000000 4.752634667103668
177100000 4.943442727637811
177200000 4.748476436682085
177300000 4.336743538211237
177400000 4.469904679368062
177500000 2.4863482007878255
177600000 4.391108679511577
177700000 4.212764644877904
177800000 4.321579819168292
177900000 4.02175373119843
178000000 4.0168884334291945
178100000 1.7235562760906387
178200000 4.109689705553939
178300000 4.078113814150771
178400000 4.9091144877172015
178500000 4.225956314181715
178600000 4.154641146880719
178700000 2.5276548572538227
178800000 4.395769488370329
178900000 4.386906592806498
179000000 4.517013349838584
179100000 4.628749563741892
179200000 3.5989899469931657
179300000 2.9139598877353534
179400000 4.505834532298565
179500000 4.116785999768141
179600000 4.070275578707384
179700000 4.0006732789222195
179800000 3.154722611329297
179900000 2.4770312591988546
180000000 4.010058648664037
180100000 4.

205400000 2.294548184174296
205500000 1.4516978995400367
205600000 1.2689256764507626
205700000 2.466866823856248
205800000 2.410973452013552
205900000 2.5418782968042395
206000000 2.431387799865087
206100000 2.559194093213548
206200000 2.2856115647442805
206300000 2.3730007579396135
206400000 1.352765969985187
206500000 1.8946999938628177
206600000 2.3587301255357267
206700000 2.342671936371525
206800000 2.3078801784917777
206900000 2.445308194494774
207000000 2.4204256658142604
207100000 2.4560535992019643
207200000 1.9587679040107462
207300000 1.4563106640267969
207400000 1.426966941834804
207500000 2.2590818235038252
207600000 2.328350827001373
207700000 2.3597308952650162
207800000 2.3918909738150336
207900000 2.1494859324708777
208000000 2.16139042867738
208100000 2.368365242355973
208200000 0.9722600894117653
208300000 2.3011603253559336
208400000 2.2076626075934604
208500000 2.1865909941724637
208600000 2.17810408093421
208700000 2.3039530057678324
208800000 2.2690121539741654


233400000 0.1277271403273642
233500000 0.1130215193227033
233600000 0.13138948805212974
233700000 0.11615597170517841
233800000 0.11945936014443637
best so far: 0
type: [1, 1, 1, 121, 1] 121
cases of this type: 1771561
233900000 0.0656662531343831
234000000 0.022164013790488244
234100000 0.021226947264720998
234200000 0.02137694935416844
234300000 0.020133066197544337
234400000 0.021339516028649277
234500000 0.020016876001828245
234600000 0.018982803720106682
234700000 0.01812768692721261
234800000 0.017456995127164657
234900000 0.018293091746836902
235000000 0.01763947216146191
235100000 0.016236584689974787
235200000 0.016350260050714015
235300000 0.01584877295248376
235400000 0.013246009833667012
235500000 0.01393918553284804
235600000 0.01367536074174775
best so far: 0
type: [1, 1, 11, 1, 11] 121
cases of this type: 1771561
235700000 0.04833463913436069
235800000 0.057815872574309514
235900000 0.04624810756230023
236000000 0.047897420745809875
236100000 0.04770242635390162
23620000